# local_files

> Local files browser route handlers

In [ ]:
#| default_exp routes.local_files

In [ ]:
#| export
from typing import Optional, Tuple, Dict, Callable

from fasthtml.common import APIRouter

from cjm_fasthtml_file_browser.core.config import FileBrowserConfig
from cjm_fasthtml_file_browser.providers.local import LocalFileSystemProvider

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.routes.core import (
    WorkflowStateStore, _get_step_state, _update_step_state
)
from cjm_transcript_source_select.components.local_files import (
    _render_local_files_browser,
    _create_db_browser_config,
    _get_file_browser_state
)
from cjm_transcript_source_select.services.source import (
    SourceService, validate_and_toggle_external_db
)

## File Browser Router Initialization

In [ ]:
#| export
# Shared provider and config instances
_local_files_provider: Optional[LocalFileSystemProvider] = None
_local_files_config: Optional[FileBrowserConfig] = None

In [ ]:
#| export
def _get_local_files_provider() -> LocalFileSystemProvider:
    """Get or create the local files provider singleton."""
    global _local_files_provider
    if _local_files_provider is None:
        _local_files_provider = LocalFileSystemProvider()
    return _local_files_provider

In [ ]:
#| export
def _handle_browse_directory(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for external db ops
    request,  # FastHTML request object
    sess,  # FastHTML session object
    path: str,  # Directory path to browse
    urls: SelectionUrls,  # URL bundle for rendering
):  # Local files browser component
    """Browse a directory and return the local files browser component."""
    session_id = get_session_id(sess)
    provider = _get_local_files_provider()
    config = _get_local_files_config()
    
    # Validate and normalize the path
    valid, error = provider.is_valid_path(path)
    if valid and provider.is_directory(path):
        normalized_path = provider.normalize_path(path)
    else:
        normalized_path = provider.get_home_path()
    
    # Get current state and update browser state
    step_state = _get_step_state(state_store, workflow_id, session_id)
    external_db_paths = step_state.get("external_db_paths", [])
    browser_state = _get_file_browser_state(step_state, provider.get_home_path())
    browser_state.current_path = normalized_path
    
    # Sync browser selection state with external_db_paths for checkbox display
    browser_state.selection.selected_paths = list(external_db_paths)
    
    # Save updated state
    _update_step_state(
        state_store, workflow_id, session_id,
        file_browser_state=browser_state.to_dict(),
        current_browse_path=normalized_path
    )
    
    return _render_local_files_browser(
        browser_state=browser_state,
        external_paths=external_db_paths,
        provider=provider,
        config=config,
        navigate_url=urls.browse_directory,
        select_url=urls.add_external,
        remove_url=urls.remove_external,
        refresh_url=urls.browse_directory,
        path_input_url=urls.browse_directory,
        home_path=provider.get_home_path(),
    )

In [ ]:
#| export
def _get_local_files_config() -> FileBrowserConfig:
    """Get or create the local files config singleton."""
    global _local_files_config
    if _local_files_config is None:
        _local_files_config = _create_db_browser_config()
    return _local_files_config

## Add/Remove External Source Handlers

In [ ]:
#| export
def _handle_add_external_source(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for external db ops
    request,  # FastHTML request object
    sess,  # FastHTML session object
    path: str,  # Path to the .db file (from file-browser select_url)
    urls: SelectionUrls,  # URL bundle for rendering
):  # Local files browser component
    """Toggle an external database source (add if not present, remove if present)."""
    session_id = get_session_id(sess)
    provider = _get_local_files_provider()
    config = _get_local_files_config()
    
    # Get current state
    step_state = _get_step_state(state_store, workflow_id, session_id)
    external_db_paths = step_state.get("external_db_paths", [])
    browser_state = _get_file_browser_state(step_state, provider.get_home_path())
    
    # Validate and toggle
    external_db_paths, error_message = validate_and_toggle_external_db(
        source_service, path, external_db_paths
    )
    _update_step_state(state_store, workflow_id, session_id, external_db_paths=external_db_paths)
    source_service.set_external_paths(external_db_paths)
    
    # Sync browser selection state with external_db_paths for checkbox display
    browser_state.selection.selected_paths = list(external_db_paths)
    _update_step_state(state_store, workflow_id, session_id, file_browser_state=browser_state.to_dict())
    
    return _render_local_files_browser(
        browser_state=browser_state,
        external_paths=external_db_paths,
        provider=provider,
        config=config,
        navigate_url=urls.browse_directory,
        select_url=urls.add_external,
        remove_url=urls.remove_external,
        refresh_url=urls.browse_directory,
        path_input_url=urls.browse_directory,
        home_path=provider.get_home_path(),
        error_message=error_message,
    )

In [ ]:
#| export
def _handle_remove_external_source(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for external db ops
    request,  # FastHTML request object
    sess,  # FastHTML session object
    db_path: str,  # Path to the .db file to remove
    urls: SelectionUrls,  # URL bundle for rendering
):  # Local files browser component
    """Remove an external database source from the Added Sources list."""
    session_id = get_session_id(sess)
    provider = _get_local_files_provider()
    config = _get_local_files_config()
    
    # Get current state
    step_state = _get_step_state(state_store, workflow_id, session_id)
    external_db_paths = step_state.get("external_db_paths", [])
    browser_state = _get_file_browser_state(step_state, provider.get_home_path())
    
    # Remove the path if it exists
    if db_path in external_db_paths:
        external_db_paths.remove(db_path)
        _update_step_state(state_store, workflow_id, session_id, external_db_paths=external_db_paths)
        source_service.set_external_paths(external_db_paths)
    
    # Sync browser selection state with external_db_paths for checkbox display
    browser_state.selection.selected_paths = list(external_db_paths)
    _update_step_state(state_store, workflow_id, session_id, file_browser_state=browser_state.to_dict())
    
    return _render_local_files_browser(
        browser_state=browser_state,
        external_paths=external_db_paths,
        provider=provider,
        config=config,
        navigate_url=urls.browse_directory,
        select_url=urls.add_external,
        remove_url=urls.remove_external,
        refresh_url=urls.browse_directory,
        path_input_url=urls.browse_directory,
        home_path=provider.get_home_path(),
    )

## Router

In [ ]:
#| export
def init_local_files_router(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for external db ops
    prefix: str,  # Route prefix (e.g., "/workflow/selection/local_files")
    urls: SelectionUrls,  # URL bundle for rendering
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize local files browser routes."""
    router = APIRouter(prefix=prefix)

    @router
    def browse_directory(request, sess, path: str):
        """Browse a directory in the local files browser."""
        return _handle_browse_directory(
            state_store, workflow_id, source_service,
            request, sess, path, urls=urls,
        )

    @router
    def add_external(request, sess, path: str):
        """Add an external database source (select_url target from file browser)."""
        return _handle_add_external_source(
            state_store, workflow_id, source_service,
            request, sess, path, urls=urls,
        )

    @router
    def remove_external(request, sess, db_path: str):
        """Remove an external database source."""
        return _handle_remove_external_source(
            state_store, workflow_id, source_service,
            request, sess, db_path, urls=urls,
        )

    routes = {
        "browse_directory": browse_directory,
        "add_external": add_external,
        "remove_external": remove_external,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()